In [1]:
import pandas

Let's create a dictionary for replacing the antibiotic 3 letter codes with their proper names; will be useful in Tables

In [2]:
drug_names_table = pandas.read_csv("dat/drugs/drug_names_lookup.csv")
drug_names_table.set_index("DRUG", inplace=True)
drug_names_lookup = {}
for idx, row in drug_names_table.iterrows():
    drug_names_lookup[idx] = row.DRUG_NAME.capitalize()
drug_names_lookup

{'AMI': 'Amikacin',
 'BDQ': 'Bedaquiline',
 'CAP': 'Capreomycin',
 'CFZ': 'Clofazimine',
 'DLM': 'Delamanid',
 'EMB': 'Ethambutol',
 'ETH': 'Ethionamide',
 'INH': 'Isoniazid',
 'KAN': 'Kanamycin',
 'LEV': 'Levofloxacin',
 'LZD': 'Linezolid',
 'MXF': 'Moxifloxacin',
 'PZA': 'Pyrazinamide',
 'RFB': 'Rifabutin',
 'RIF': 'Rifampicin',
 'STM': 'Streptomycin'}

Load in the drugs in the WHOv2 catalogue, on the UKMYC plate, those we've chosen to use MGIT samples for etc

In [3]:
who_drugs = list(pandas.read_csv('dat/drugs/who2_drugs.csv').drug)
plate_drugs = list(pandas.read_csv('dat/drugs/plate_drugs.csv').drug)
other_drugs = list(pandas.read_csv('dat/drugs/other_drugs.csv').drug)
mgit_drugs = list(pandas.read_csv('dat/drugs/mgit_drugs.csv').drug)

plate_drug_qualities = [str(i)+"_QUALITY" for i in plate_drugs]

Now load in the 1,000 samples from the CRyPTIC project and the 1,663 MGIT samples for the five drugs where we either have relatively few resistant samples or aren't present on the UKMYC plate designs

In [4]:
UKMYC = pandas.read_csv('dat/UKMYC_1000_samples.csv')
UKMYC.set_index(['ENA_RUN_ACCESSION', 'UNIQUEID'], inplace=True)
UKMYC[:2]

,,ENA_STUDY_ACCESSION,ENA_SAMPLE_ACCESSION,HAS_ALL_DRUGS,N_SUSCEPTIBLE,N_RESISTANT,N_HIGH_QUALITY,AMI,BDQ,CFZ,DLM,...,EMB_QUALITY,ETH_QUALITY,INH_QUALITY,KAN_QUALITY,LEV_QUALITY,LZD_QUALITY,MXF_QUALITY,RIF_QUALITY,FASTQ_FTP,POS_AVG_GROWTH
ENA_RUN_ACCESSION,UNIQUEID,,,,,,,,,,,,,,,,,,,,,
ERR4810791,site.02.subj.0068.lab.22A018.iso.1,PRJEB41194,SAMEA7542366,True,6,6,9,S,S,R,S,...,HIGH,HIGH,HIGH,HIGH,LOW,LOW,HIGH,HIGH,ftp.sra.ebi.ac.uk/vol1/fastq/ERR481/001/ERR481...,44.68
ERR4810943,site.02.subj.0091.lab.22A043.iso.1,PRJEB41194,SAMEA7544003,True,5,7,11,S,S,S,S,...,HIGH,HIGH,HIGH,HIGH,HIGH,LOW,HIGH,HIGH,ftp.sra.ebi.ac.uk/vol1/fastq/ERR481/003/ERR481...,43.23


In [18]:
UKMYC.columns

Index(['ENA_STUDY_ACCESSION', 'ENA_SAMPLE_ACCESSION', 'HAS_ALL_DRUGS',
       'N_SUSCEPTIBLE', 'N_RESISTANT', 'N_HIGH_QUALITY', 'AMI', 'BDQ', 'CFZ',
       'DLM', 'EMB', 'ETH', 'INH', 'KAN', 'LEV', 'LZD', 'MXF', 'RIF',
       'AMI_QUALITY', 'BDQ_QUALITY', 'CFZ_QUALITY', 'DLM_QUALITY',
       'EMB_QUALITY', 'ETH_QUALITY', 'INH_QUALITY', 'KAN_QUALITY',
       'LEV_QUALITY', 'LZD_QUALITY', 'MXF_QUALITY', 'RIF_QUALITY', 'FASTQ_FTP',
       'POS_AVG_GROWTH'],
      dtype='object', name='DRUG')

In [5]:
MGIT = pandas.read_csv('dat/MGIT_1663_samples.csv')
MGIT.set_index(['ENA_RUN_ACCESSION','UNIQUEID'], inplace=True)
MGIT[:2]

,,WCHG_ID,ENA_SAMPLE_ACCESSION,ENA_STUDY_ACCESSION,HAS_ALL_DRUGS,N_SUSCEPTIBLE,N_RESISTANT,N_DRUGS,BDQ,CAP,LZD,PZA,STM,FASTQ_FTP
ENA_RUN_ACCESSION,UNIQUEID,,,,,,,,,,,,,
ERR9992611,site.10.subj.BC02023023.lab.BC02023023.iso.1,WTCHG_933274_70255001,SAMEA110419968,PRJEB55007,False,0,2,2,R,NaN,R,NaN,NaN,ftp.sra.ebi.ac.uk/vol1/fastq/ERR999/001/ERR999...
ERR13289349,site.10.subj.BH02818474.lab.BH02818474.iso.1,WTCHG_934880_70665042,SAMEA115747815,PRJEB76547,False,1,1,2,S,NaN,R,NaN,NaN,ftp.sra.ebi.ac.uk/vol1/fastq/ERR132/049/ERR132...


Let's make some bash scripts to make it easier to download all the FASTQ files for processing

In [6]:
OUTPUT = open('dat/UKMYC_1000_samples_download.sh', 'w')
for idx,row in UKMYC.iterrows():
    paths = row.FASTQ_FTP.split(';')
    for i in paths:
        OUTPUT.write('wget '+i+'\n')
OUTPUT.close()

OUTPUT = open('dat/MGIT_1663_samples_download.sh', 'w')
for idx,row in MGIT.iterrows():
    paths = row.FASTQ_FTP.split(';')
    for i in paths:
        OUTPUT.write('wget '+i+'\n')
OUTPUT.close()

Create a table summarising the proportion of resistant samples by drug in this dataset

In [7]:
def find_position(df, drugs):
    current_position = {}
    for i in drugs:
        current_position[i] = []
        for phenotype in ['R', 'S']:
            current_position[i].append((df[i]==phenotype).sum())
    
    df = pandas.DataFrame.from_dict(current_position,orient='index',columns=['R','S'])
    df.sort_values(by='R',ascending=False, inplace=True)
    df['TOTAL'] = df['R'] + df['S']
    df['%R'] = (df['R'] / df['TOTAL'] * 100)
    return df

summary_ukmyc = find_position(UKMYC, plate_drugs)
summary_ukmyc['%R'] = summary_ukmyc['%R'].map('{:,.1f}'.format)
summary_ukmyc['Phenotype'] = 'UKMYC'
summary_ukmyc

,R,S,TOTAL,%R,Phenotype
INH,500,500,1000,50.0,UKMYC
RIF,476,524,1000,47.6,UKMYC
LEV,323,677,1000,32.3,UKMYC
KAN,289,711,1000,28.9,UKMYC
EMB,288,712,1000,28.8,UKMYC
MXF,288,712,1000,28.8,UKMYC
CFZ,288,712,1000,28.8,UKMYC
AMI,288,712,1000,28.8,UKMYC
ETH,288,712,1000,28.8,UKMYC
DLM,140,860,1000,14.0,UKMYC


Repeat for the MGIT samples (remembering that each sample has a varying number of MGIT results which we will describe shortly)

In [8]:
summary_mgit = find_position(MGIT, mgit_drugs)
summary_mgit['%R'] = summary_mgit['%R'].map('{:,.1f}'.format)
summary_mgit['Phenotype'] = 'MGIT'
summary_mgit

,R,S,TOTAL,%R,Phenotype
BDQ,394,380,774,50.9,MGIT
PZA,291,336,627,46.4,MGIT
CAP,256,575,831,30.8,MGIT
STM,252,362,614,41.0,MGIT
LZD,105,675,780,13.5,MGIT


In [9]:
summary = pandas.concat([summary_ukmyc, summary_mgit])
summary = summary[['Phenotype', '%R', 'R', 'S', 'TOTAL']]
summary

,Phenotype,%R,R,S,TOTAL
INH,UKMYC,50.0,500,500,1000
RIF,UKMYC,47.6,476,524,1000
LEV,UKMYC,32.3,323,677,1000
KAN,UKMYC,28.9,289,711,1000
EMB,UKMYC,28.8,288,712,1000
MXF,UKMYC,28.8,288,712,1000
CFZ,UKMYC,28.8,288,712,1000
AMI,UKMYC,28.8,288,712,1000
ETH,UKMYC,28.8,288,712,1000
DLM,UKMYC,14.0,140,860,1000


Finally, let's prepare this table for incorporation in the LaTeX manuscript

In [10]:
nice_summary = summary.rename(drug_names_lookup)
print(nice_summary.to_latex())

\begin{tabular}{lllrrr}
\toprule
 & Phenotype & %R & R & S & TOTAL \\
\midrule
Isoniazid & UKMYC & 50.0 & 500 & 500 & 1000 \\
Rifampicin & UKMYC & 47.6 & 476 & 524 & 1000 \\
Levofloxacin & UKMYC & 32.3 & 323 & 677 & 1000 \\
Kanamycin & UKMYC & 28.9 & 289 & 711 & 1000 \\
Ethambutol & UKMYC & 28.8 & 288 & 712 & 1000 \\
Moxifloxacin & UKMYC & 28.8 & 288 & 712 & 1000 \\
Clofazimine & UKMYC & 28.8 & 288 & 712 & 1000 \\
Amikacin & UKMYC & 28.8 & 288 & 712 & 1000 \\
Ethionamide & UKMYC & 28.8 & 288 & 712 & 1000 \\
Delamanid & UKMYC & 14.0 & 140 & 860 & 1000 \\
Linezolid & UKMYC & 11.7 & 117 & 883 & 1000 \\
Bedaquiline & UKMYC & 6.5 & 65 & 935 & 1000 \\
Bedaquiline & MGIT & 50.9 & 394 & 380 & 774 \\
Pyrazinamide & MGIT & 46.4 & 291 & 336 & 627 \\
Capreomycin & MGIT & 30.8 & 256 & 575 & 831 \\
Streptomycin & MGIT & 41.0 & 252 & 362 & 614 \\
Linezolid & MGIT & 13.5 & 105 & 675 & 780 \\
\bottomrule
\end{tabular}



To be able to join to the results from the genetics we need to move from a "short, wide" format to a "long, thin" format which will also let us combine both datasets into one. First the UKMYC data 

In [11]:
UKMYC.columns.name='DRUG'
plate_samples_phenotypes = UKMYC[plate_drugs].stack().to_frame(name='BINARY_PHENOTYPE')
plate_samples_qualities = UKMYC[plate_drug_qualities]
plate_samples_qualities.rename(columns={str(i)+"_QUALITY":str(i) for i in plate_drugs}, inplace=True)
plate_samples_qualities = plate_samples_qualities[plate_drugs].stack().to_frame(name='PHENOTYPE_QUALITY')
plate_samples_phenotypes = plate_samples_phenotypes.join(plate_samples_qualities)
plate_samples_phenotypes['PHENOTYPE_METHOD'] = 'UKMYC'
plate_samples_phenotypes[:3]

/var/folders/84/9nb2gsqj4hlcxztvwf19wg680000gq/T/ipykernel_10428/1620681778.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plate_samples_qualities.rename(columns={str(i)+"_QUALITY":str(i) for i in plate_drugs}, inplace=True)


BINARY_PHENOTYPE  \
ENA_RUN_ACCESSION UNIQUEID                           DRUG                    
ERR4810791        site.02.subj.0068.lab.22A018.iso.1 INH                 R   
                                                     RIF                 R   
                                                     EMB                 R   

                                                          PHENOTYPE_QUALITY  \
ENA_RUN_ACCESSION UNIQUEID                           DRUG                     
ERR4810791        site.02.subj.0068.lab.22A018.iso.1 INH               HIGH   
                                                     RIF               HIGH   
                                                     EMB               HIGH   

                                                          PHENOTYPE_METHOD  
ENA_RUN_ACCESSION UNIQUEID                           DRUG                   
ERR4810791        site.02.subj.0068.lab.22A018.iso.1 INH             UKMYC  
                                                     RIF             UKMYC  
                                                     EMB             UKMYC

..and now the MGIT data

In [12]:
MGIT.columns.name='DRUG'
mgit_samples_phenotypes = MGIT[mgit_drugs].stack().to_frame(name='BINARY_PHENOTYPE')
mgit_samples_phenotypes['PHENOTYPE_QUALITY'] = None
mgit_samples_phenotypes['PHENOTYPE_METHOD'] = 'MGIT'
mgit_samples_phenotypes[:3]

BINARY_PHENOTYPE  \
ENA_RUN_ACCESSION UNIQUEID                                     DRUG                    
ERR9992611        site.10.subj.BC02023023.lab.BC02023023.iso.1 LZD                 R   
                                                               BDQ                 R   
ERR13289349       site.10.subj.BH02818474.lab.BH02818474.iso.1 LZD                 R   

                                                                    PHENOTYPE_QUALITY  \
ENA_RUN_ACCESSION UNIQUEID                                     DRUG                     
ERR9992611        site.10.subj.BC02023023.lab.BC02023023.iso.1 LZD               None   
                                                               BDQ               None   
ERR13289349       site.10.subj.BH02818474.lab.BH02818474.iso.1 LZD               None   

                                                                    PHENOTYPE_METHOD  
ENA_RUN_ACCESSION UNIQUEID                                     DRUG                   
ERR9992611        site.10.subj.BC02023023.lab.BC02023023.iso.1 LZD              MGIT  
                                                               BDQ              MGIT  
ERR13289349       site.10.subj.BH02818474.lab.BH02818474.iso.1 LZD              MGIT

Before we go on, let's take a look at how many MGIT results each sample has

In [13]:
mgit_samples_summary = mgit_samples_phenotypes.reset_index()
mgit_samples_summary = mgit_samples_summary[['ENA_RUN_ACCESSION', 'DRUG']].groupby('ENA_RUN_ACCESSION').count() #.sort_values(by='ENA_RUN_ACCESSION', ascending=False)
mgit_samples_summary.rename(columns={'DRUG':'DRUG_COUNT'}, inplace=True)
mgit_samples_summary = mgit_samples_summary.DRUG_COUNT.value_counts().sort_index()
mgit_samples_summary = pandas.DataFrame(mgit_samples_summary)
mgit_samples_summary

,count
DRUG_COUNT,
1,347
2,702
3,581
4,33


In [14]:
print(mgit_samples_summary.to_latex())

\begin{tabular}{lr}
\toprule
 & count \\
DRUG_COUNT &  \\
\midrule
1 & 347 \\
2 & 702 \\
3 & 581 \\
4 & 33 \\
\bottomrule
\end{tabular}



Now we can concatenate these two datasets to make analysis easier later on

In [15]:
phenotypes = pandas.concat([mgit_samples_phenotypes, plate_samples_phenotypes])
phenotypes

BINARY_PHENOTYPE  \
ENA_RUN_ACCESSION UNIQUEID                                     DRUG                    
ERR9992611        site.10.subj.BC02023023.lab.BC02023023.iso.1 LZD                 R   
                                                               BDQ                 R   
ERR13289349       site.10.subj.BH02818474.lab.BH02818474.iso.1 LZD                 R   
                                                               BDQ                 S   
ERR9992662        site.10.subj.IF02701910.lab.IF02701910.iso.1 LZD                 R   
...                                                                              ...   
ERR4812280        site.05.subj.CA-0634.lab.CO-12310-18.iso.1   CFZ                 S   
                                                               DLM                 S   
                                                               AMI                 S   
                                                               ETH                 S   
                                                               KAN                 S   

                                                                    PHENOTYPE_QUALITY  \
ENA_RUN_ACCESSION UNIQUEID                                     DRUG                     
ERR9992611        site.10.subj.BC02023023.lab.BC02023023.iso.1 LZD               None   
                                                               BDQ               None   
ERR13289349       site.10.subj.BH02818474.lab.BH02818474.iso.1 LZD               None   
                                                               BDQ               None   
ERR9992662        site.10.subj.IF02701910.lab.IF02701910.iso.1 LZD               None   
...                                                                               ...   
ERR4812280        site.05.subj.CA-0634.lab.CO-12310-18.iso.1   CFZ               HIGH   
                                                               DLM               HIGH   
                                                               AMI               HIGH   
                                                               ETH               HIGH   
                                                               KAN               HIGH   

                                                                    PHENOTYPE_METHOD  
ENA_RUN_ACCESSION UNIQUEID                                     DRUG                   
ERR9992611        site.10.subj.BC02023023.lab.BC02023023.iso.1 LZD              MGIT  
                                                               BDQ              MGIT  
ERR13289349       site.10.subj.BH02818474.lab.BH02818474.iso.1 LZD              MGIT  
                                                               BDQ              MGIT  
ERR9992662        site.10.subj.IF02701910.lab.IF02701910.iso.1 LZD              MGIT  
...                                                                              ...  
ERR4812280        site.05.subj.CA-0634.lab.CO-12310-18.iso.1   CFZ             UKMYC  
                                                               DLM             UKMYC  
                                                               AMI             UKMYC  
                                                               ETH             UKMYC  
                                                               KAN             UKMYC  

[15626 rows x 3 columns]

Be paranoid and check there are no duplicated rows as a result of combining these two datasets

In [16]:
assert phenotypes.index.duplicated().sum()==0

Finally write the table to disc for later

In [17]:
phenotypes.to_csv('dat/PHENOTYPES.csv')

In [20]:
plate_samples_phenotypes.reset_index(inplace=True)
plate_samples_phenotypes[:2]

,ENA_RUN_ACCESSION,UNIQUEID,DRUG,BINARY_PHENOTYPE,PHENOTYPE_QUALITY,PHENOTYPE_METHOD
0,ERR4810791,site.02.subj.0068.lab.22A018.iso.1,INH,R,HIGH,UKMYC
1,ERR4810791,site.02.subj.0068.lab.22A018.iso.1,RIF,R,HIGH,UKMYC


In [21]:
plate_samples_phenotypes.set_index(['UNIQUEID', 'DRUG'], inplace=True)
plate_samples_phenotypes[:2]

ENA_RUN_ACCESSION BINARY_PHENOTYPE  \
UNIQUEID                           DRUG                                      
site.02.subj.0068.lab.22A018.iso.1 INH         ERR4810791                R   
                                   RIF         ERR4810791                R   

                                        PHENOTYPE_QUALITY PHENOTYPE_METHOD  
UNIQUEID                           DRUG                                     
site.02.subj.0068.lab.22A018.iso.1 INH               HIGH            UKMYC  
                                   RIF               HIGH            UKMYC

In [22]:
DST_MEASUREMENTS = pandas.read_pickle('/Users/fowler/files/cryptic/cryptic-tables-v2.1.2/DST_MEASUREMENTS.pkl.gz')
DST_MEASUREMENTS = DST_MEASUREMENTS[DST_MEASUREMENTS['METHOD_3'].isin(['UKMYC5','UKMYC6'])]
DST_MEASUREMENTS[:3]

SOURCE      METHOD_1  \
UNIQUEID                              DRUG                               
site.27.subj.HCMC2001.lab.17760.iso.1 BDQ   SEQTREAT2020  liquid media   
                                      KAN   SEQTREAT2020  liquid media   
                                      ETH   SEQTREAT2020  liquid media   

                                                       METHOD_2 METHOD_3  \
UNIQUEID                              DRUG                                 
site.27.subj.HCMC2001.lab.17760.iso.1 BDQ   microdilution plate   UKMYC5   
                                      KAN   microdilution plate   UKMYC5   
                                      ETH   microdilution plate   UKMYC5   

                                           METHOD_CC METHOD_MIC PHENOTYPE  \
UNIQUEID                              DRUG                                  
site.27.subj.HCMC2001.lab.17760.iso.1 BDQ        NaN        NaN         U   
                                      KAN        NaN        NaN         U   
                                      ETH        NaN        NaN         U   

                                           QUALITY  
UNIQUEID                              DRUG          
site.27.subj.HCMC2001.lab.17760.iso.1 BDQ   MEDIUM  
                                      KAN   MEDIUM  
                                      ETH   MEDIUM

In [30]:
df = plate_samples_phenotypes.join(DST_MEASUREMENTS[['METHOD_3','METHOD_MIC']], how='left')
df.rename(columns={'METHOD_3':'PLATEDESIGN', 'METHOD_MIC':'MIC'}, inplace=True)
df.reset_index(inplace=True)
df.set_index(['PLATEDESIGN','DRUG','MIC'], inplace=True)
df

UNIQUEID  \
PLATEDESIGN DRUG MIC                                                   
UKMYC5      INH  >1.6             site.02.subj.0068.lab.22A018.iso.1   
            RIF  >4               site.02.subj.0068.lab.22A018.iso.1   
            EMB  >8               site.02.subj.0068.lab.22A018.iso.1   
            BDQ  0.06             site.02.subj.0068.lab.22A018.iso.1   
            LZD  2.0              site.02.subj.0068.lab.22A018.iso.1   
...                                                              ...   
UKMYC6      CFZ  <=0.03   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
            DLM  <=0.008  site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
            AMI  <=0.25   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
            ETH  0.5      site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
            KAN  <=1      site.05.subj.CA-0634.lab.CO-12310-18.iso.1   

                         ENA_RUN_ACCESSION BINARY_PHENOTYPE PHENOTYPE_QUALITY  \
PLATEDESIGN DRUG MIC                                                            
UKMYC5      INH  >1.6           ERR4810791                R              HIGH   
            RIF  >4             ERR4810791                R              HIGH   
            EMB  >8             ERR4810791                R              HIGH   
            BDQ  0.06           ERR4810791                S              HIGH   
            LZD  2.0            ERR4810791                R               LOW   
...                                    ...              ...               ...   
UKMYC6      CFZ  <=0.03         ERR4812280                S              HIGH   
            DLM  <=0.008        ERR4812280                S              HIGH   
            AMI  <=0.25         ERR4812280                S              HIGH   
            ETH  0.5            ERR4812280                S              HIGH   
            KAN  <=1            ERR4812280                S              HIGH   

                         PHENOTYPE_METHOD  
PLATEDESIGN DRUG MIC                       
UKMYC5      INH  >1.6               UKMYC  
            RIF  >4                 UKMYC  
            EMB  >8                 UKMYC  
            BDQ  0.06               UKMYC  
            LZD  2.0                UKMYC  
...                                   ...  
UKMYC6      CFZ  <=0.03             UKMYC  
            DLM  <=0.008            UKMYC  
            AMI  <=0.25             UKMYC  
            ETH  0.5                UKMYC  
            KAN  <=1                UKMYC  

[12000 rows x 5 columns]

In [31]:
PLATE_LAYOUT = pandas.read_pickle('/Users/fowler/files/cryptic/cryptic-tables-v2.1.2/PLATE_LAYOUT.pkl.gz')
PLATE_LAYOUT.reset_index(inplace=True)
PLATE_LAYOUT.rename(columns={'CONC':'MIC'}, inplace=True)
PLATE_LAYOUT.set_index(['PLATEDESIGN','DRUG','MIC'], inplace=True)
PLATE_LAYOUT[:3]

DILUTION  ROW  COL BINARY_PHENOTYPE
PLATEDESIGN DRUG MIC                                     
UKMYC5      AMI  >8          7  NaN  NaN                R
                 8.0         6  1.0  1.0                R
                 4.0         5  2.0  1.0                R

In [32]:
df = df.join(PLATE_LAYOUT[['DILUTION']], how='inner')
df.DILUTION.value_counts(dropna=False).sort_index()

DILUTION
1     2133
2     2130
3     1959
4     1428
5     1127
6     1044
7      795
8      987
9      137
10     217
11      43
Name: count, dtype: int64

In [34]:
df.reset_index(inplace=True)

In [35]:
df

,PLATEDESIGN,DRUG,MIC,UNIQUEID,ENA_RUN_ACCESSION,BINARY_PHENOTYPE,PHENOTYPE_QUALITY,PHENOTYPE_METHOD,DILUTION
0,UKMYC5,INH,>1.6,site.02.subj.0068.lab.22A018.iso.1,ERR4810791,R,HIGH,UKMYC,8
1,UKMYC5,RIF,>4,site.02.subj.0068.lab.22A018.iso.1,ERR4810791,R,HIGH,UKMYC,8
2,UKMYC5,EMB,>8,site.02.subj.0068.lab.22A018.iso.1,ERR4810791,R,HIGH,UKMYC,9
3,UKMYC5,BDQ,0.06,site.02.subj.0068.lab.22A018.iso.1,ERR4810791,S,HIGH,UKMYC,3
4,UKMYC5,LZD,2.0,site.02.subj.0068.lab.22A018.iso.1,ERR4810791,R,LOW,UKMYC,7
...,...,...,...,...,...,...,...,...,...
11995,UKMYC6,CFZ,<=0.03,site.05.subj.CA-0634.lab.CO-12310-18.iso.1,ERR4812280,S,HIGH,UKMYC,1
11996,UKMYC6,DLM,<=0.008,site.05.subj.CA-0634.lab.CO-12310-18.iso.1,ERR4812280,S,HIGH,UKMYC,1
11997,UKMYC6,AMI,<=0.25,site.05.subj.CA-0634.lab.CO-12310-18.iso.1,ERR4812280,S,HIGH,UKMYC,1
11998,UKMYC6,ETH,0.5,site.05.subj.CA-0634.lab.CO-12310-18.iso.1,ERR4812280,S,HIGH,UKMYC,2


In [36]:
df.columns

Index(['PLATEDESIGN', 'DRUG', 'MIC', 'UNIQUEID', 'ENA_RUN_ACCESSION',
       'BINARY_PHENOTYPE', 'PHENOTYPE_QUALITY', 'PHENOTYPE_METHOD',
       'DILUTION'],
      dtype='object')

In [37]:
df.set_index(['ENA_RUN_ACCESSION','DRUG'], inplace=True)
df

PLATEDESIGN      MIC  \
ENA_RUN_ACCESSION DRUG                        
ERR4810791        INH       UKMYC5     >1.6   
                  RIF       UKMYC5       >4   
                  EMB       UKMYC5       >8   
                  BDQ       UKMYC5     0.06   
                  LZD       UKMYC5      2.0   
...                            ...      ...   
ERR4812280        CFZ       UKMYC6   <=0.03   
                  DLM       UKMYC6  <=0.008   
                  AMI       UKMYC6   <=0.25   
                  ETH       UKMYC6      0.5   
                  KAN       UKMYC6      <=1   

                                                          UNIQUEID  \
ENA_RUN_ACCESSION DRUG                                               
ERR4810791        INH           site.02.subj.0068.lab.22A018.iso.1   
                  RIF           site.02.subj.0068.lab.22A018.iso.1   
                  EMB           site.02.subj.0068.lab.22A018.iso.1   
                  BDQ           site.02.subj.0068.lab.22A018.iso.1   
                  LZD           site.02.subj.0068.lab.22A018.iso.1   
...                                                            ...   
ERR4812280        CFZ   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
                  DLM   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
                  AMI   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
                  ETH   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
                  KAN   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   

                       BINARY_PHENOTYPE PHENOTYPE_QUALITY PHENOTYPE_METHOD  \
ENA_RUN_ACCESSION DRUG                                                       
ERR4810791        INH                 R              HIGH            UKMYC   
                  RIF                 R              HIGH            UKMYC   
                  EMB                 R              HIGH            UKMYC   
                  BDQ                 S              HIGH            UKMYC   
                  LZD                 R               LOW            UKMYC   
...                                 ...               ...              ...   
ERR4812280        CFZ                 S              HIGH            UKMYC   
                  DLM                 S              HIGH            UKMYC   
                  AMI                 S              HIGH            UKMYC   
                  ETH                 S              HIGH            UKMYC   
                  KAN                 S              HIGH            UKMYC   

                        DILUTION  
ENA_RUN_ACCESSION DRUG            
ERR4810791        INH          8  
                  RIF          8  
                  EMB          9  
                  BDQ          3  
                  LZD          7  
...                          ...  
ERR4812280        CFZ          1  
                  DLM          1  
                  AMI          1  
                  ETH          2  
                  KAN          1  

[12000 rows x 7 columns]

In [38]:
df = df[['UNIQUEID', 'BINARY_PHENOTYPE', 'PHENOTYPE_QUALITY', 'PHENOTYPE_METHOD', 'PLATEDESIGN', 'MIC', 'DILUTION' ]]
df

UNIQUEID  \
ENA_RUN_ACCESSION DRUG                                               
ERR4810791        INH           site.02.subj.0068.lab.22A018.iso.1   
                  RIF           site.02.subj.0068.lab.22A018.iso.1   
                  EMB           site.02.subj.0068.lab.22A018.iso.1   
                  BDQ           site.02.subj.0068.lab.22A018.iso.1   
                  LZD           site.02.subj.0068.lab.22A018.iso.1   
...                                                            ...   
ERR4812280        CFZ   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
                  DLM   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
                  AMI   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
                  ETH   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
                  KAN   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   

                       BINARY_PHENOTYPE PHENOTYPE_QUALITY PHENOTYPE_METHOD  \
ENA_RUN_ACCESSION DRUG                                                       
ERR4810791        INH                 R              HIGH            UKMYC   
                  RIF                 R              HIGH            UKMYC   
                  EMB                 R              HIGH            UKMYC   
                  BDQ                 S              HIGH            UKMYC   
                  LZD                 R               LOW            UKMYC   
...                                 ...               ...              ...   
ERR4812280        CFZ                 S              HIGH            UKMYC   
                  DLM                 S              HIGH            UKMYC   
                  AMI                 S              HIGH            UKMYC   
                  ETH                 S              HIGH            UKMYC   
                  KAN                 S              HIGH            UKMYC   

                       PLATEDESIGN      MIC  DILUTION  
ENA_RUN_ACCESSION DRUG                                 
ERR4810791        INH       UKMYC5     >1.6         8  
                  RIF       UKMYC5       >4         8  
                  EMB       UKMYC5       >8         9  
                  BDQ       UKMYC5     0.06         3  
                  LZD       UKMYC5      2.0         7  
...                            ...      ...       ...  
ERR4812280        CFZ       UKMYC6   <=0.03         1  
                  DLM       UKMYC6  <=0.008         1  
                  AMI       UKMYC6   <=0.25         1  
                  ETH       UKMYC6      0.5         2  
                  KAN       UKMYC6      <=1         1  

[12000 rows x 7 columns]

In [39]:
df.to_csv('dat/UKMYC_1000_phenotypes.csv')
df

UNIQUEID  \
ENA_RUN_ACCESSION DRUG                                               
ERR4810791        INH           site.02.subj.0068.lab.22A018.iso.1   
                  RIF           site.02.subj.0068.lab.22A018.iso.1   
                  EMB           site.02.subj.0068.lab.22A018.iso.1   
                  BDQ           site.02.subj.0068.lab.22A018.iso.1   
                  LZD           site.02.subj.0068.lab.22A018.iso.1   
...                                                            ...   
ERR4812280        CFZ   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
                  DLM   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
                  AMI   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
                  ETH   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   
                  KAN   site.05.subj.CA-0634.lab.CO-12310-18.iso.1   

                       BINARY_PHENOTYPE PHENOTYPE_QUALITY PHENOTYPE_METHOD  \
ENA_RUN_ACCESSION DRUG                                                       
ERR4810791        INH                 R              HIGH            UKMYC   
                  RIF                 R              HIGH            UKMYC   
                  EMB                 R              HIGH            UKMYC   
                  BDQ                 S              HIGH            UKMYC   
                  LZD                 R               LOW            UKMYC   
...                                 ...               ...              ...   
ERR4812280        CFZ                 S              HIGH            UKMYC   
                  DLM                 S              HIGH            UKMYC   
                  AMI                 S              HIGH            UKMYC   
                  ETH                 S              HIGH            UKMYC   
                  KAN                 S              HIGH            UKMYC   

                       PLATEDESIGN      MIC  DILUTION  
ENA_RUN_ACCESSION DRUG                                 
ERR4810791        INH       UKMYC5     >1.6         8  
                  RIF       UKMYC5       >4         8  
                  EMB       UKMYC5       >8         9  
                  BDQ       UKMYC5     0.06         3  
                  LZD       UKMYC5      2.0         7  
...                            ...      ...       ...  
ERR4812280        CFZ       UKMYC6   <=0.03         1  
                  DLM       UKMYC6  <=0.008         1  
                  AMI       UKMYC6   <=0.25         1  
                  ETH       UKMYC6      0.5         2  
                  KAN       UKMYC6      <=1         1  

[12000 rows x 7 columns]